In [1]:
import pickle
import tensorflow as tf
from tensorflow.contrib.layers import flatten

import numpy as np

# Visualizations will be shown in the notebook.
%matplotlib inline

# Load Camera calibration params
X_train, y_train, X_valid, y_valid, X_test, y_test = pickle.load(open( "pickle_data/prep_data.p", "rb" ) )
globalData = pickle.load(open( "pickle_data/gData.p", "rb" ) )

C:\ProgramData\Anaconda3\envs\tf\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\ProgramData\Anaconda3\envs\tf\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\ProgramData\Anaconda3\envs\tf\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\ProgramData\Anaconda3\envs\tf\lib\site-packages\tensorflow\python\framework\dty

*** 
## Step 3: Model Architecture

### Model Hyperparameters

In [2]:
EPOCHS = 30
BATCH_SIZE = 128

In [3]:
def LeNet(x):    
    # Arguments used for tf.truncated_normal, randomly defines variables for the weights and biases for each layer
    mu = 0
    sigma = 0.1
    dropout_keep_prob = 1   
    
    # Layer 1: Convolutional. Input = 32x32x3. Output = 28x28x8.
    conv1_W = tf.Variable(tf.truncated_normal(shape=(5, 5, 3, 9), mean = mu, stddev = sigma))
    conv1_b = tf.Variable(tf.zeros(9))
    conv1   = tf.nn.conv2d(x, conv1_W, strides=[1, 1, 1, 1], padding='VALID') + conv1_b
    
    # Dropout.
    conv1 = tf.nn.dropout(conv1, dropout_keep_prob)

    # Activation.
    conv1 = tf.nn.relu(conv1)

    # Pooling. Input = 28x28x9 Output = 14x14x18
    conv1 = tf.nn.max_pool(conv1, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='VALID')

    # Layer 2: Convolutional. Output = 10x10x16.
    conv2_W = tf.Variable(tf.truncated_normal(shape=(5, 5, 9, 20), mean = mu, stddev = sigma))
    conv2_b = tf.Variable(tf.zeros(20))
    conv2   = tf.nn.conv2d(conv1, conv2_W, strides=[1, 1, 1, 1], padding='VALID') + conv2_b
    
    # Dropout.
    conv2 = tf.nn.dropout(conv2, dropout_keep_prob)
    
    # Activation.
    conv2 = tf.nn.relu(conv2)

    # Pooling. Input = 10x10x16. Output = 5x5x18.
    conv2 = tf.nn.max_pool(conv2, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='VALID')

    # Flatten. Input = 5x5x12. Output = 400.
    fc0   = flatten(conv2)
    
    # Layer 3: Fully Connected. Input = 400. Output = 120.
    fc1_W = tf.Variable(tf.truncated_normal(shape=(500, 220), mean = mu, stddev = sigma))
    fc1_b = tf.Variable(tf.zeros(220))
    fc1   = tf.matmul(fc0, fc1_W) + fc1_b
    
    # Dropout.
    fc1 = tf.nn.dropout(fc1, dropout_keep_prob)
    
    # Activation.
    fc1    = tf.nn.relu(fc1)

    # Layer 4: Fully Connected. Input = 225. Output = 84.
    fc2_W  = tf.Variable(tf.truncated_normal(shape=(220, 120), mean = mu, stddev = sigma))
    fc2_b  = tf.Variable(tf.zeros(120))
    fc2    = tf.matmul(fc1, fc2_W) + fc2_b
    
    # Dropout.
    fc2 = tf.nn.dropout(fc2, dropout_keep_prob)
    
    # Activation.
    fc2    = tf.nn.relu(fc2)

    # Layer 5: Fully Connected. Input = 84. Output = 43.
    fc3_W  = tf.Variable(tf.truncated_normal(shape=(120, globalData['n_clasees']), mean = mu, stddev = sigma))
    fc3_b  = tf.Variable(tf.zeros(globalData['n_clasees']))
    logits = tf.matmul(fc2, fc3_W) + fc3_b
    
    return logits

*** 
## Step 4: Train + Validate + Test

A validation set can be used to assess how well the model is performing. A low accuracy on the training and validation
sets imply underfitting. A high accuracy on the training set but low accuracy on the validation set implies overfitting.

In [4]:
### Model training Hyperparameters
rate = 0.001

In [5]:
x = tf.placeholder(tf.float32, (None, globalData['image_shape'][0] , globalData['image_shape'][1], globalData['image_shape'][2]))
y = tf.placeholder(tf.int32, (None))
one_hot_y = tf.one_hot(y, globalData['n_clasees'])

In [6]:
logits = LeNet(x)
cross_entropy = tf.nn.softmax_cross_entropy_with_logits(labels=one_hot_y, logits=logits)
loss_operation = tf.reduce_mean(cross_entropy)
optimizer = tf.train.AdamOptimizer(learning_rate = rate)
training_operation = optimizer.minimize(loss_operation)

Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Instructions for updating:
Use keras.layers.flatten instead.
Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See `tf.nn.softmax_cross_entropy_with_logits_v2`.



In [7]:
correct_prediction = tf.equal(tf.argmax(logits, 1), tf.argmax(one_hot_y, 1))
accuracy_operation = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
saver = tf.train.Saver()

def evaluate(X_data, y_data):
    num_examples = len(X_data)
    total_accuracy = 0
    sess = tf.get_default_session()
    for offset in range(0, num_examples, BATCH_SIZE):
        batch_x, batch_y = X_data[offset:offset+BATCH_SIZE], y_data[offset:offset+BATCH_SIZE]
        accuracy = sess.run(accuracy_operation, feed_dict={x: batch_x, y: batch_y})
        total_accuracy += (accuracy * len(batch_x))
    return 100 * total_accuracy / num_examples

In [8]:
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    num_examples = len(X_train)
    
    print("Training...")
    print()
    for i in range(EPOCHS):
        X_train, y_train = X_train, y_train
        for offset in range(0, num_examples, BATCH_SIZE):
            end = offset + BATCH_SIZE
            batch_x, batch_y = X_train[offset:end], y_train[offset:end]
            sess.run(training_operation, feed_dict={x: batch_x, y: batch_y})
            
        training_accuracy = evaluate(X_train, y_train)
        validation_accuracy = evaluate(X_valid, y_valid)
        print("EPOCH {} ...".format(i+1))
        print("Training Accuracy = {:2.2f} %".format(training_accuracy))
        print("Validation Accuracy = {:2.2f} %".format(validation_accuracy))
        print("Diff Accuracy = {:2.2f} %".format(training_accuracy-validation_accuracy))
        print()
        
    saver.save(sess, './lenet')
    print("Model saved")

Training...

EPOCH 1 ...
Training Accuracy = 92.20 %
Validation Accuracy = 83.61 %
Diff Accuracy = 8.60 %

EPOCH 2 ...
Training Accuracy = 96.93 %
Validation Accuracy = 88.82 %
Diff Accuracy = 8.10 %

EPOCH 3 ...
Training Accuracy = 98.17 %
Validation Accuracy = 89.77 %
Diff Accuracy = 8.40 %

EPOCH 4 ...
Training Accuracy = 98.28 %
Validation Accuracy = 89.95 %
Diff Accuracy = 8.32 %

EPOCH 5 ...
Training Accuracy = 98.97 %
Validation Accuracy = 90.36 %
Diff Accuracy = 8.61 %

EPOCH 6 ...
Training Accuracy = 99.20 %
Validation Accuracy = 91.04 %
Diff Accuracy = 8.16 %

EPOCH 7 ...
Training Accuracy = 99.01 %
Validation Accuracy = 91.47 %
Diff Accuracy = 7.54 %

EPOCH 8 ...
Training Accuracy = 98.82 %
Validation Accuracy = 91.02 %
Diff Accuracy = 7.80 %

EPOCH 9 ...
Training Accuracy = 99.41 %
Validation Accuracy = 91.81 %
Diff Accuracy = 7.59 %

EPOCH 10 ...
Training Accuracy = 98.51 %
Validation Accuracy = 91.43 %
Diff Accuracy = 7.08 %

EPOCH 11 ...
Training Accuracy = 99.56 %
Valid

In [8]:
with tf.Session() as sess:
    saver.restore(sess, './lenet')
    testing_accuracy = evaluate(X_test, y_test)
    print("Testing Accuracy = {:.3f} %".format(testing_accuracy))
    print()

Instructions for updating:
Use standard file APIs to check for files with this prefix.
INFO:tensorflow:Restoring parameters from ./lenet
Testing Accuracy = 93.325 %



In [10]:
pickle.dump( saver, open( "pickle_data/NeuralNetwork.p", "wb" ) )

TypeError: can't pickle _thread.RLock objects